<a href="https://colab.research.google.com/github/Tarunkumar314/accenture_hackthon/blob/main/acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Step 1: Install required libraries
!pip install -q pandas

# Step 2: Import libraries
import pandas as pd
import sqlite3
from google.colab import files

# Step 3: Upload CSV file
print("Please upload your job_description.csv file")
uploaded = files.upload()

# Step 4: Load CSV with safe encoding
df = pd.read_csv("job_description.csv", encoding='ISO-8859-1')
print("✅ File loaded successfully!\n")

# Step 5: View the top 5 rows
print("📌 Preview of the data:")
print(df.head())

# Step 6: Display column names
print("\n📌 Column names:")
print(df.columns)

# Step 7: Save to SQLite database
conn = sqlite3.connect("job_data.db")
df.to_sql("job_descriptions", conn, if_exists="replace", index=False)
conn.close()

print("\n✅ Data successfully saved to SQLite database named 'job_data.db'")


Please upload your job_description.csv file


Saving job_description.csv to job_description (3).csv
✅ File loaded successfully!

📌 Preview of the data:
               Job Title                                    Job Description  \
0      Software Engineer   Description:\nWe are seeking a skilled Softwa...   
1         Data Scientist  Job Description:\nWe are looking for a skilled...   
2        Product Manager  Description:\nWe are seeking an innovative and...   
3         Cloud Engineer  Description:\nWe are looking for a skilled Clo...   
4  Cybersecurity Analyst  Description:\nWe are looking for a skilled Cyb...   

  Unnamed: 2  
0             
1        NaN  
2        NaN  
3        NaN  
4        NaN  

📌 Column names:
Index(['Job Title', 'Job Description', 'Unnamed: 2'], dtype='object')

✅ Data successfully saved to SQLite database named 'job_data.db'


In [6]:
# Step 8: Clean job descriptions
df['Job Description'] = df['Job Description'].str.replace(r"(?i)description:\s*", "", regex=True)
df['Job Description'] = df['Job Description'].str.replace(r"\n", " ", regex=True)

# Display cleaned version
print("✅ Cleaned job descriptions:")
print(df['Job Description'].head())


✅ Cleaned job descriptions:
0     We are seeking a skilled Software Engineer to...
1    Job We are looking for a skilled Data Scientis...
2    We are seeking an innovative and strategic Pro...
3    We are looking for a skilled Cloud Engineer to...
4    We are looking for a skilled Cybersecurity Ana...
Name: Job Description, dtype: object


In [7]:
# Step 9: Install transformers and summarization model
!pip install -q transformers sentencepiece

from transformers import pipeline

# Load summarizer (Flan-T5)
summarizer = pipeline("summarization", model="google/flan-t5-small", tokenizer="google/flan-t5-small")

# Summarize each JD (truncated if too long for model)
def summarize_text(text):
    try:
        return summarizer(text[:512], max_length=60, min_length=20, do_sample=False)[0]['summary_text']
    except:
        return "Summary failed"

df['Summary'] = df['Job Description'].apply(summarize_text)

# Show summaries
print("✅ Summarized JDs:")
print(df[['Job Title', 'Summary']].head())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


✅ Summarized JDs:
               Job Title                                            Summary
0      Software Engineer  We are seeking a skilled Software Engineer to ...
1         Data Scientist  Data Scientist to analyze complex datasets, de...
2        Product Manager  Product Manager to lead the development and ex...
3         Cloud Engineer  Cloud Engineer to design, implement, and manag...
4  Cybersecurity Analyst  Cybersecurity Analyst is a skilled cybersecuri...


In [8]:
# Save updated DataFrame (with summaries) into database
conn = sqlite3.connect("job_data.db")
df.to_sql("job_descriptions", conn, if_exists="replace", index=False)
conn.close()

print("\n✅ Summaries saved to SQLite database.")



✅ Summaries saved to SQLite database.


In [11]:
from google.colab import files

print("📂 Upload your CV PDFs (you can select multiple files)")
uploaded = files.upload()


📂 Upload your CV PDFs (you can select multiple files)


Saving C1161.pdf to C1161.pdf


In [12]:
!pip install -q PyMuPDF
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# Create a dictionary with filenames and their extracted text
cv_texts = {}
for filename in uploaded:
    cv_texts[filename] = extract_text_from_pdf(filename)

# Preview one CV's extracted text
first_cv = list(cv_texts.items())[0]
print(f"\n📄 Extracted text from {first_cv[0]}:\n")
print(first_cv[1][:1000])  # Print only first 1000 characters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.9 MB/s eta 0:00:00

📄 Extracted text from C1161.pdf:

Candidate Resume (ID: C1161)
Name: Richard Molina
Email: richardmolina72@gmail.com
Phone: +1-908-7110
Education
Bachelor of Engineering in Information Technology (2014-2018)
Concentrated on database management, networking, and cybersecurity.
Master of Business Administration (2017-2019)
Focused on Business Strategy, Financial Analysis, and Operations Management.
Bachelor of Science in Computer Science (2015-2019)
Specialized in Software Development and AI, with projects in deep learning and cloud computing.
Work Experience
Product Manager at DEF Ltd. (2017-2021)
Led cross-functional teams to develop innovative solutions, increasing product adoption by 40%.
Software Engineer at XYZ Corp (2018-2022)
Developed scalable backend applications, improved system efficiency by 30%, and led agile
development sprints.
Skills
Python & Machine Learning - Proficient in TensorFlow, PyTorch

In [13]:
import re

def extract_cv_sections(cv_text):
    sections = {
        "name": None,
        "email": None,
        "education": [],
        "experience": [],
        "skills": [],
        "certifications": []
    }

    # Extract name
    name_match = re.search(r"Name:\s*(.+)", cv_text)
    sections["name"] = name_match.group(1).strip() if name_match else None

    # Extract email
    email_match = re.search(r"Email:\s*(\S+@\S+)", cv_text)
    sections["email"] = email_match.group(1).strip() if email_match else None

    # Extract education
    edu_match = re.findall(r"(Bachelor|Master).+?\((\d{4}-\d{4})\)", cv_text)
    sections["education"] = [" ".join(match) for match in edu_match]

    # Extract work experience
    exp_match = re.findall(r"(?<=at\s).+?\((\d{4}-\d{4})\)", cv_text)
    job_titles = re.findall(r"(?<=\n)[A-Z][a-z]+\s(?:Manager|Engineer|Analyst|Developer)", cv_text)
    sections["experience"] = [f"{title} ({exp})" for title, exp in zip(job_titles, exp_match)]

    # Extract skills
    skill_match = re.search(r"Skills\s*(.*?)(?=Certifications|$)", cv_text, re.S)
    if skill_match:
        skills_block = skill_match.group(1)
        sections["skills"] = [s.strip() for s in re.split(r"[-•]", skills_block) if len(s.strip()) > 2]

    # Extract certifications
    cert_match = re.search(r"Certifications\s*(.*)", cv_text, re.S)
    if cert_match:
        certifications_block = cert_match.group(1)
        sections["certifications"] = [s.strip() for s in certifications_block.splitlines() if s.strip()]

    return sections

cv_data = extract_cv_sections(first_cv[1])
for k, v in cv_data.items():
    print(f"🔹 {k.capitalize()}: {v}")


🔹 Name: Richard Molina
🔹 Email: richardmolina72@gmail.com
🔹 Education: ['Bachelor 2014-2018', 'Master 2017-2019', 'Bachelor 2015-2019']
🔹 Experience: ['Product Manager (2017-2021)', 'Software Engineer (2018-2022)']
🔹 Skills: ['Python & Machine Learning', 'Proficient in TensorFlow, PyTorch, and Scikit', 'learn with hands', 'on\nexperience in deploying AI solutions.']
🔹 Certifications: ['AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS', 'solutions, optimizing performance and security.', 'Achievements', 'Developed an AI chatbot - Built a chatbot that reduced customer support tickets by 40%, enhancing', 'user experience and efficiency.', 'Tech Stack', 'Java, Spring Boot, MySQL, Kafka, Azure DevOps']


In [14]:
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util

# Load lightweight but powerful model
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 844.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# Combine candidate sections
candidate_profile = " ".join(cv_data['education'] + cv_data['experience'] + cv_data['skills'] + cv_data['certifications'])

# Pick a sample job description from the DataFrame
jd_text = df['Job Description'].iloc[0]  # You can loop through all later

# Generate embeddings
cv_embedding = model.encode(candidate_profile, convert_to_tensor=True)
jd_embedding = model.encode(jd_text, convert_to_tensor=True)

# Compute similarity
similarity_score = util.pytorch_cos_sim(cv_embedding, jd_embedding).item()
print(f"\n📊 Semantic Similarity Score with JD 1: {similarity_score:.4f}")



📊 Semantic Similarity Score with JD 1: 0.6258


In [16]:
if similarity_score >= 0.8:
    print("✅ Candidate shortlisted for interview!")
else:
    print("❌ Candidate does not meet the threshold.")


❌ Candidate does not meet the threshold.


In [17]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Load JD DataFrame (already loaded)
df['embedding'] = df['Job Description'].apply(lambda x: model.encode(x))

# Save JD embeddings to SQLite
conn = sqlite3.connect('job_data.db')
df.to_sql('job_descriptions', conn, if_exists='replace', index=False)
conn.close()


In [18]:
import numpy as np

# Reload JD embeddings from DB
conn = sqlite3.connect('job_data.db')
jd_df = pd.read_sql("SELECT * FROM job_descriptions", conn)
conn.close()

# Generate embedding for candidate
cv_text = " ".join(cv_data['education'] + cv_data['experience'] + cv_data['skills'] + cv_data['certifications'])
cv_embed = model.encode(cv_text)

# Compute similarity scores
scores = [util.pytorch_cos_sim(cv_embed, np.array(eval(embed))).item() for embed in jd_df['embedding']]

# Add match scores to DataFrame
jd_df['match_score'] = scores
shortlisted = jd_df[jd_df['match_score'] >= 0.8]


SyntaxError: source code string cannot contain null bytes (<string>)